In [29]:
%pip install fuzzywuzzy
%pip install Levenshtein
%pip install yahoo-finance
%pip install yfinance
%pip install -U nbformat
%pip install scipy



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   - -------------------------------------- 1.3/41.0 MB 11.2 MB/s eta 0:00:04
   ------ --------------------------------- 6.6/41.0 M

In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
import yfinance as yf
import plotly.express as px
from functools import lru_cache

In [3]:
df1 = pd.read_csv('./stock_part1.csv')
df2 = pd.read_csv('./stock_part2.csv')

df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ID           200000 non-null  int64 
 1   Date         200000 non-null  object
 2   Ticker       200000 non-null  object
 3   Open         200000 non-null  object
 4   High         200000 non-null  object
 5   Low          200000 non-null  object
 6   Close        200000 non-null  object
 7   Adj Close    200000 non-null  object
 8   Volume       200000 non-null  object
 9   Dividend     200000 non-null  object
 10  Split Ratio  161520 non-null  object
dtypes: int64(1), object(10)
memory usage: 16.8+ MB


In [15]:
df1

,ID,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Dividend,Split Ratio
0,1,2023-09-03,meta,258.44,909.63,1380.05,"595,86",61.54,"8,846,040,350",3.57,1:1
1,2,"Aug 28, 2024",NVDA,678.96,725.03,185.51,295.83,843.78,7346935699,2.85,3:2
2,3,"May 10, 2022",INTU,1348.94,215.00,831.86,874.05,"$336,19","8,027,589,419",2.28,2:1
3,4,2017-12-26,NVDA,$186.39,1111.85,508.65,511.49,$952.63,1939.5M,4.51,3:2
4,5,"Jan 20, 2022",WORK,1425.94,692.20,340.75,$378.58,1493.44,8984007687,1.93,NaN
...,...,...,...,...,...,...,...,...,...,...,...
199995,199996,"Jul 27, 2019",TSLA,"269,17",144.45,"418,85",588.19,$893.89,4184.54M,"4,71",3:2
199996,199997,"Sep 13, 2021",VMW,315.78,262.78,"89,13","$172,09",659.42,"5,782,458,138",1.04,3:2
199997,199998,2016-08-19,aapl,327.66,897.77,"1402,05",140.90,904.11,3104296586,$3.21,NaN
199998,199999,2020-06-24,DJCU,141.99,739.41,285.76,$529.92,1201.64,45454067,1.51,NaN


In [4]:

df1['Date_original'] = df1['Date']

df1['Date'] = pd.to_datetime(df1['Date'], format="mixed", errors='coerce')


invalid_dates = df1[df1['Date'].isna()]

invalid_values = df1.loc[invalid_dates.index, 'Date_original']


nerr = len(invalid_dates) # quantas datas invalidas tem, ex: 12/63/2000 ou 12/81/2023
percent = nerr / len(df1["Date"]) * 100
print(f"Percentual de erros: {percent}%")
print(f"Número de erros: {nerr}")

#invalid_values.to_csv('datas_invalidas.csv')

Percentual de erros: 1.015%
Número de erros: 2030


In [5]:
from datetime import datetime

future_dates = df1[df1['Date'] > datetime.now()]["Date"]
print(datetime.now())


#future_dates.to_csv("datas_futuro.csv")

2025-05-21 21:20:24.160927


In [11]:
maiormes = 0
for i in future_dates.dt.strftime('%Y-%m-%d'):
    mesatual = int(i[-2:])
    maiormes = ( mesatual if mesatual > maiormes else maiormes)
print(maiormes)

5


In [12]:
future_dates = future_dates.apply(lambda x: x.replace(day=x.month, month=x.day) if pd.notnull(x) else x)

df1.loc[future_dates.index, "Date"] = future_dates


In [8]:
df1["Ticker"] = df1["Ticker"].str.upper().str.strip()

ticker_counts = df1["Ticker"].value_counts()
unique_ticker = ticker_counts[ticker_counts > 10].index
unique_values = df1[df1["Ticker"].isin(unique_ticker)]
print (unique_ticker)

Index(['ZM', 'SQ', 'PYPL', 'SHOP', 'META', 'BABA', 'SNAP', 'WORK', 'AMD',
       'CSCO', 'ORCL', 'AAPL', 'INTC', 'TSLA', 'NVDA', 'LYFT', 'IBM', 'SPOT',
       'VMW', 'ADBE', 'TWTR', 'PINS', 'MSFT', 'INTU', 'GOOG', 'DOCU', 'NFLX',
       'CRM', 'AMZN', 'UBER', 'INTF', 'EMD'],
      dtype='object', name='Ticker')


In [13]:
ticker_counts = df1['Ticker'].value_counts()
print(ticker_counts.describe())

count    3307.000000
mean       60.477774
std       551.331473
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max      6161.000000
Name: count, dtype: float64


In [ ]:
#Acho melhor baixar um df com os tickers válidos, para não precisar fazer requisição na api do yahoo
df1["Ticker"] = df1["Ticker"].str.upper().str.strip()


ticker_counts = df1["Ticker"].value_counts()


valid_tickers = list(set(
    ticker_counts[ticker_counts > 10].index.tolist()
))
valid_tickers = [t.upper() for t in valid_tickers]


@lru_cache(maxsize=None)
def is_valid_ticker(ticker):
    try:
        t = yf.Ticker(ticker)
        hist = t.history(period="1d")  
        return not hist.empty
    except:
        return False


def get_fuzzy_match(ticker, candidates, min_score=65):
    if not ticker or len(ticker) < 2:
        return None
    candidates_filtered = [t for t in candidates if abs(len(t) - len(ticker)) <= 1]
    if not candidates_filtered:
        return None
    match, score = process.extractOne(ticker, candidates_filtered)
    if score >= min_score:
        return match
    return None


corrections = {}
for ticker in ticker_counts[ticker_counts <= 10].index:
    if not is_valid_ticker(ticker):
        corrected = get_fuzzy_match(ticker, valid_tickers)
        if corrected:
            corrections[ticker] = corrected

print("Correções feitas:")
for k, v in corrections.items():
    print(f"{k} → {v}")


df1["Ticker"] = df1["Ticker"].replace(corrections)

$RBM: possibly delisted; no price data found  (period=1d)
$BMZN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$INTB: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$AMZB: possibly delisted; no price data found  (period=1d)
$DOCG: possibly delisted; no price data found  (period=1d)
$IDM: possibly delisted; no price data found  (period=1d)
$IAPL: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$ADFE: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$AFZN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$AMW: possibly delisted; no price data found  (period=1d)
$UBJR: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$AMZ: possibly del

In [14]:
numerics_columns = ["Open", "High", "Low", "Close", "Adj Close", "Volume", "Dividend"]
def get_non_numeric_characters(column):
  print(column.name)
  tot = set("")
  for index, value in column.items():
    tot = tot | set([c for c in value if not c.isdigit()])
  print(tot)

def count_comma(column):
    count = 0
    for index,value in column.items():
        if isinstance(value, str) and len(value) >= 3:
            if value[-3] in [',', '.']:
                count += 1
    print(count)



for i in numerics_columns:
  get_non_numeric_characters(df1[i])
  count_comma(df1[i])



Open
{'.', ',', 'Z', '$'}
200000
High
{'.', ',', 'Z', '$'}
200000
Low
{'.', ',', 'Z', '$'}
200000
Close
{'.', ',', 'Z', '$'}
200000
Adj Close
{'.', ',', 'Z', '$'}
200000
Volume
{'K', 'n', '.', 'l', 'i', 'b', 'B', 'M', 'o', ' ', ','}
3503
Dividend
{'.', ',', 'Z', '$'}
200000


In [15]:
equivalents_numeric_columns = numerics_columns
equivalents_numeric_columns.remove("Volume")

def change_caracter (value):
  value = value.strip("$")
  value = value.strip()
  value = value.replace("Z", "2")
  value = value.replace(".", ",")
  return value

for i in equivalents_numeric_columns:
  df1[i] = df1[i].apply(change_caracter)

In [16]:
for i in equivalents_numeric_columns:
    df1[i] = df1[i].apply(
        lambda value: value[:-3] + "." + value[-2:] if len(value) >= 3 else value
    )
    df1[i] = df1[i].astype(float)   

In [17]:
df1

,ID,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Dividend,Split Ratio,Date_original
0,1,2023-09-03,meta,258.44,909.63,1380.05,595.86,61.54,"8,846,040,350",3.57,1:1,2023-09-03
1,2,2024-08-28,NVDA,678.96,725.03,185.51,295.83,843.78,7346935699,2.85,3:2,"Aug 28, 2024"
2,3,2022-05-10,INTU,1348.94,215.00,831.86,874.05,336.19,"8,027,589,419",2.28,2:1,"May 10, 2022"
3,4,2017-12-26,NVDA,186.39,1111.85,508.65,511.49,952.63,1939.5M,4.51,3:2,2017-12-26
4,5,2022-01-20,WORK,1425.94,692.20,340.75,378.58,1493.44,8984007687,1.93,NaN,"Jan 20, 2022"
...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199996,2019-07-27,TSLA,269.17,144.45,418.85,588.19,893.89,4184.54M,4.71,3:2,"Jul 27, 2019"
199996,199997,2021-09-13,VMW,315.78,262.78,89.13,172.09,659.42,"5,782,458,138",1.04,3:2,"Sep 13, 2021"
199997,199998,2016-08-19,aapl,327.66,897.77,1402.05,140.90,904.11,3104296586,3.21,NaN,2016-08-19
199998,199999,2020-06-24,DJCU,141.99,739.41,285.76,529.92,1201.64,45454067,1.51,NaN,2020-06-24


In [63]:
def encontrar_data(ticker, open_val, close_val):
    t = yf.Ticker(ticker)
    hist = t.history(period="max")

    resultado = hist[
        (hist['Open'].round(5) == round(open_val, 2)) &
        (hist['Close'].round(5) == round(close_val, 2))
        
    ]
    if not resultado.empty:
        return resultado.index[0].date()  # ou .strftime('%Y-%m-%d')
    return None

celtest = df1.iloc[135]
print(encontrar_data(celtest["Ticker"], celtest["Open"], celtest["Close"]))

print(celtest)
# Exemplo de uso:
# data_encontrada = encontrar_data('AAPL', 150.00, 155.00, 1000000)

None
ID                      136
Date                    NaT
Ticker                 PINS
Open                 473.86
High                 741.34
Low                  245.77
Close                1129.4
Adj Close           1134.51
Volume           3710049432
Dividend               3.96
Split Ratio             2:1
Date_original     4/63/2024
Name: 135, dtype: object


In [ ]:
from datetime import timedelta

def get_operacao_por_data_ticker(ticker, data):
    t = yf.Ticker(ticker)
    
    # Garantir que a data está como datetime
    data_dt = datetime.strptime(data, "%Y-%m-%d")
    data_fim = data_dt + timedelta(days=1)

    hist = t.history(start=data_dt.strftime("%Y-%m-%d"), end=data_fim.strftime("%Y-%m-%d"))
    
    if not hist.empty:
        
        return hist.iloc[0][['Open', 'High', 'Low', 'Close', 'Volume']]
    else:
        return None


valores = get_operacao_por_data_ticker('NVDA', '2024-08-28')
print(valores)


t = yf.Ticker('NVDA')
print(t.history(period="5d"))


Open      1.280884e+02
High      1.282984e+02
Low       1.226098e+02
Close     1.255791e+02
Volume    4.481011e+08
Name: 2024-08-28 00:00:00-04:00, dtype: float64
                                 Open        High         Low       Close  \
Date                                                                        
2025-05-15 00:00:00-04:00  134.289993  136.300003  132.660004  134.830002   
2025-05-16 00:00:00-04:00  136.220001  136.350006  133.460007  135.399994   
2025-05-19 00:00:00-04:00  132.389999  135.869995  132.389999  135.570007   
2025-05-20 00:00:00-04:00  134.289993  134.580002  132.619995  134.380005   
2025-05-21 00:00:00-04:00  133.044998  137.399994  130.589996  131.800003   

                              Volume  Dividends  Stock Splits  
Date                                                           
2025-05-15 00:00:00-04:00  226632600        0.0           0.0  
2025-05-16 00:00:00-04:00  226542500        0.0           0.0  
2025-05-19 00:00:00-04:00  193154600     

In [ ]:
# Exemplo: contar quantos valores em 'coluna_a' são maiores que em 'coluna_b'
count = (df1['High'] > df1['Open']).sum()
print(count)
print((count/len(df1))*100)




from scipy.stats import gaussian_kde


x = df1['High'].dropna()  
kde = gaussian_kde(x)
x_range = np.linspace(x.min(), x.max(), 500)  
y_density = kde(x_range)  


fig = px.line(x=x_range, y=y_density, title='Curva de Densidade dos Valores', labels={'x': 'High', 'y': 'Densidade'})
fig.show()

99588
49.794


In [32]:
# {'K', 'n', '.', 'l', 'i', 'b', 'B', 'M', 'o', ' ', ','}


get_non_numeric_characters(df1["Volume"].apply( lambda x: x.strip()))

Volume
{'K', 'n', '.', 'l', 'i', 'b', 'B', 'M', 'o', ' ', ','}
